In [42]:
import os.path
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from mlab_func import spect_gen
import scipy.stats
from typing import Tuple
%matplotlib qt

In [43]:
class BearingPlot:
    def __init__(self, ds_name: str, data_path: str):
        ## Initial Parameters
        if dataset_name == 'IMS':
            self.Fs = 20000
            self.N = 20480
            self.t = np.linspace(0, self.N / self.Fs, self.N)
            self.ShaftSpeed = 2000 / 60
            self.BPFI = 8.908 * self.ShaftSpeed
            self.BPFO = 7.092 * self.ShaftSpeed
            self.BSF = 4.197 * self.ShaftSpeed
            self.data_path = os.path.join(data_path, '2nd_test-4sensor.mat')
        else:
            self.Fs = 25600
            self.N = 2560
            self.t = np.linspace(0, 0.1, self.N)
            self.REBcode = 'Ax 1_1'
            self.ShaftSpeed = 1800 / 60
            self.BPFI = 7.389 * self.ShaftSpeed
            self.BPFO = 5.611 * self.ShaftSpeed
            self.BSF = 3.589 * self.ShaftSpeed
            self.data_path = os.path.join(data_path, self.REBcode + '.mat')
        self._load_data()

    def _load_data(self):
        ## Load data
        data_bearing = scipy.io.loadmat(self.data_path)

        if dataset_name == 'IMS':
            sensor_type = 'S1'
        else:
            sensor_type = 'ACCx'

        self.VibData = data_bearing[sensor_type]
        self.Life = self.VibData[0, :]
        self.Acc = self.VibData[1:, :]
    def draw_signal_plots(self, signal_nums: Tuple):
        ## TDF Calculation and presentation
        # RMS
        RMS = np.sqrt(np.mean(self.Acc ** 2, axis=0))
        fig = plt.figure(1, figsize=(15, 10))
        fig.clear()
        rms_ax = plt.subplot(3, 2, 1)
        plt.ion()
        plt.plot(RMS, 'y')
        plt.grid(True)
        plt.xlabel('Signal Number')
        plt.ylabel('RMS & Peak (g)')

        # Peak
        peak = np.max(np.abs(self.Acc), axis=0)
        plt.plot(peak, 'r')
        plt.grid(True)
        plt.xlabel('Signal Number')
        plt.ylabel('RMS & Peak (g)')
        plt.legend(['RMS', 'Peak'], loc='best', prop={'size': 12})

        # Kurtosis
        kurt = scipy.stats.kurtosis(self.Acc)
        kurt_ax = plt.subplot(3, 2, 3)
        plt.plot(kurt)
        plt.grid(True)
        plt.xlabel('Signal Number')
        plt.ylabel('Kurtosis')

        #Crest Factor
        CF = peak / RMS
        cf_ax = plt.subplot(3, 2, 5)
        plt.plot(CF)
        plt.grid(True)
        plt.xlabel('Signal Number')
        plt.ylabel('CF')

        amplitude_ax = plt.subplot(3, 2, 2)
        spect_gen_ax = plt.subplot(3, 2, 4)
        freq_scaled_ax = plt.subplot(3, 2, 6)

        i = 0
        for x in signal_nums:
            i += 1
            a = int(np.floor(i / 4))
            c = np.abs(2.1 - a)
            b = i - a * 4
            rms_ax.scatter(x, peak[x], color=np.array([1 - b / 4, b / 4, 1 - b / 4]), linewidth=2, zorder=10)
            kurt_ax.scatter(x, kurt[x], color=np.array([1 - b / 4, b / 4, 1 - b / 4]), linewidth=2, zorder=10)
            cf_ax.scatter(x, CF[x], color=np.array([1 - b / 4, b / 4, 1 - b / 4]), linewidth=2, zorder=10)

            ## Time wave and Spectrum presentation
            current_signal = self.Acc[:, x]
            amplitude_ax.plot(self.t, current_signal, color=np.array([1 - b / 4, b / 4, 1 - b / 4]), linewidth=0.5)
            amplitude_ax.grid(True)
            amplitude_ax.set_xlabel('t (s)')
            amplitude_ax.set_ylabel('Amplitude (g)')

            Y, f = spect_gen(current_signal, self.Fs)
            spect_gen_ax.plot(f, Y, color=np.array([1 - b / 4, b / 4, 1 - b / 4]))
            spect_gen_ax.set_title('FFT of Vibration signal')
            spect_gen_ax.set_xlabel('Frequency (Hz)')
            spect_gen_ax.set_ylabel('Amplitude (g)')
            spect_gen_ax.set_xlim(np.array([0, 1000]))

            freq_scaled_ax.plot(f, Y, color=np.array([1 - b / 4, b / 4, 1 - b / 4]))
            freq_scaled_ax.set_title('FFT of Vibration signal')
            freq_scaled_ax.set_xlabel('Frequency (Hz)')
            freq_scaled_ax.set_ylabel('Amplitude (g)')
            freq_scaled_ax.grid(True)
            freq_scaled_ax.set_xlim(np.array([0, self.Fs / 2.5]))

            #Plot the characteristic frequencies
            x_bounds = spect_gen_ax.get_xlim()
            spect_gen_ax.axvline(self.BPFI, c='y', ls='--', label='BPFI')
            spect_gen_ax.annotate(text='BPFI', xy=(((self.BPFI - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                  xycoords='axes fraction', verticalalignment='top', horizontalalignment='left', rotation=270,
                                  fontsize=12, color='y')
            spect_gen_ax.axvline(2 * self.BPFI, c='y', ls='--', label='2xBPFI')
            spect_gen_ax.annotate(text='2xBPFI', xy=(((2 * self.BPFI - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                  xycoords='axes fraction', verticalalignment='top', horizontalalignment='left', rotation=270,
                                  fontsize=12, color='y')
            spect_gen_ax.axvline(3 * self.BPFI, c='y', ls='--', label='3xBPFI')
            spect_gen_ax.annotate(text='3xBPFI', xy=(((3 * self.BPFI - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                  xycoords='axes fraction', verticalalignment='top', horizontalalignment='left', rotation=270,
                                  fontsize=12, color='y')
            spect_gen_ax.axvline(self.BSF, c='g', ls='--', label='BSF')
            spect_gen_ax.annotate(text='BSF', xy=(((self.BSF - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                  xycoords='axes fraction', verticalalignment='top', horizontalalignment='left', rotation=270,
                                  fontsize=12, color='g')
            spect_gen_ax.axvline(2 * self.BSF, c='g', ls='--', label='2xBSF')
            spect_gen_ax.annotate(text='2xBSF', xy=(((2 * self.BSF - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                  xycoords='axes fraction', verticalalignment='top', horizontalalignment='left', rotation=270,
                                  fontsize=12, color='g')
            spect_gen_ax.axvline(3 * self.BSF, c='g', ls='--', label='3xBSF')
            spect_gen_ax.annotate(text='3xBSF', xy=(((3 * self.BSF - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                  xycoords='axes fraction', verticalalignment='top', horizontalalignment='left', rotation=270,
                                  fontsize=12, color='g')
            spect_gen_ax.axvline(self.BPFO, c='r', ls='--', label='BPFO')
            spect_gen_ax.annotate(text='BPFO', xy=(((self.BPFO - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                  xycoords='axes fraction', verticalalignment='top', horizontalalignment='left', rotation=270,
                                  fontsize=12, color='r')
            spect_gen_ax.axvline(2 * self.BPFO, c='r', ls='--', label='2xBPFO')
            spect_gen_ax.annotate(text='2xBPFO', xy=(((2 * self.BPFO - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                  xycoords='axes fraction', verticalalignment='top', horizontalalignment='left', rotation=270,
                                  fontsize=12, color='r')
            spect_gen_ax.axvline(3 * self.BPFO, c='r', ls='--', label='3xBPFO')
            spect_gen_ax.annotate(text='3xBPFO', xy=(((3 * self.BPFO - x_bounds[0]) / (x_bounds[1] - x_bounds[0])), 1.01),
                                  xycoords='axes fraction', verticalalignment='top', horizontalalignment='left', rotation=270,
                                  fontsize=12, color='r')

        legends = ['RMS', 'Peak'] + list(signal_nums)
        rms_ax.legend(legends)
        fig.tight_layout()
        plt.draw_all()

In [44]:
dataset_name = 'PRONOSTIA'  #@param ['IMS', 'PRONOSTIA']
data_path = '/ssd/Code/iran-bearing/data/'

# signals = (0, 10, 100, 200, 500, 750)
signals = (1800,)

bearing_plot = BearingPlot(ds_name=dataset_name, data_path=data_path)
bearing_plot.draw_signal_plots(signals)